<a href="https://colab.research.google.com/github/vijayshankarrealdeal/GoogleColab/blob/main/gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [471]:
from google.colab import drive

In [472]:
# drive.mount("gdrive")

In [473]:
path = '/content/gdrive/MyDrive/Dataset'
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets  as tfds
import re

In [474]:
dataset = pd.read_csv(path+'/wgnd_ctry.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [475]:
dataset = dataset.drop(['gchar12','gchar1','gchar2'],axis = 1)

In [476]:
import matplotlib.pyplot as plt

In [477]:
def convertFM(data):
  if data == 'F':
    return 0
  else:
    return 1

In [478]:
dataset.gender = dataset.gender.apply(lambda x: convertFM(x))

In [479]:
dataset = dataset[dataset.code != 'KR']
dataset = dataset[dataset.code != 'CN']

In [480]:
df = dataset[dataset.code == 'IN']
dff = dataset[dataset.code == 'AM']
dfff = dataset[dataset.code == 'US']

In [481]:
new_dataset = pd.concat([df,dff,dfff])

In [482]:
new_dataset.drop(['code'],inplace=True,axis = 1)

In [483]:
new_dataset

,name,gender
61,AADHYA,0
448,AAMOD,1
449,AAMOD AAMOD,1
450,AAMOD DINKAR,1
513,AANYA,0
...,...,...
289448,ZYTAVION,1
289449,ZYTAVIOUS,1
289451,ZYVION,1
289453,ZYYANNA,0


In [484]:
X = new_dataset.name

In [485]:
y = new_dataset.gender.values

In [486]:
def token_stem(S):
  S = S.lower()
  S = re.sub('[^a-zA-Z]',' ',str(S))
  S = re.sub(r"^\s+",'',S)
  S = ''.join(S)
  # S = np.array([str(k) for k in S])
  return S

In [487]:
X = X.apply(lambda x:token_stem(str(x)))


In [488]:
X = X.values

In [489]:
X

array(['aadhya', 'aamod', 'aamod aamod', ..., 'zyvion', 'zyyanna',
       'zzyzx'], dtype=object)

In [490]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    X,target_vocab_size = 2**16
)

In [491]:
tokenizer.save_to_file("hello")

In [492]:
data_input = [tokenizer.encode(sentences) for sentences in X]

In [493]:
Max_len = max([len(u) for u in data_input])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    data_input,
            value = 0,
            padding = "post",
            maxlen = Max_len
)


In [494]:
Max_len

8

In [495]:
data_inputs[0]

array([15374,     0,     0,     0,     0,     0,     0,     0],
      dtype=int32)

In [496]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data_inputs,y,test_size = 0.30,random_state = 0)

In [497]:
x_train.shape

(70017, 8)

In [498]:
class DeepCNN(tf.keras.Model):
    def __init__(self,
                 vocab_size,
                 emb_dim = 128,
                 nb_filter = 50,
                 FNN_units = 512,
                 nb_classes = 2,
                 dropout_rate  = 0.1,
                 training = False,
                 name = 'deep_cnn'
                ):
        super(DeepCNN,self).__init__(name = name)
        
        self.embedding = tf.keras.layers.Embedding(vocab_size,emb_dim)
        #define three layers of filters
        self.bigram = tf.keras.layers.Conv1D(filters = nb_filter,
                                             kernel_size = 2,
                                             padding = "valid",
                                             activation = 'relu'
                                             )
        self.pool_1= tf.keras.layers.GlobalMaxPool1D() 

        self.trigram = tf.keras.layers.Conv1D(filters = nb_filter,
                                              kernel_size = 3,
                                             padding = "valid",
                                             activation = 'relu'
                                             )
        self.pool_2= tf.keras.layers.GlobalMaxPool1D() 

        self.fourgram = tf.keras.layers.Conv1D(filters = nb_filter,
                                               kernel_size = 4,
                                             padding = "valid",
                                             activation = 'relu'
                                             )
        self.pool_3= tf.keras.layers.GlobalMaxPool1D() 

        self.dense_1 = tf.keras.layers.Dense(units = FNN_units,
                                             activation='relu')
        
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

        self.last_dense = tf.keras.layers.Dense(units = 1,
                                                activation='sigmoid')

    def call(self,inputs,training):
      x = self.embedding(inputs)
      x_1 = self.bigram(x)
      x_1 = self.pool_1(x_1)

      x_2 = self.trigram(x)
      x_2 = self.pool_1(x_2)

      x_3 = self.fourgram(x)
      x_3 = self.pool_1(x_3)


      merged = tf.concat([x_1,x_2,x_3],axis = -1)
      merged = self.dense_1(merged)
      merged = self.dropout(merged,training)

      output = self.last_dense(merged)

      return output

In [499]:
VOCAB_SIZE = tokenizer.vocab_size
EMB_DIM = 200
FILTERS = 100
FNN_UNITS = 256
BATCH_SIZE = 32
DROPOUT_RATE = 0.2
EPOCHS = 3
NB_CLASSES = 2

In [500]:
cnn = DeepCNN(vocab_size=VOCAB_SIZE,
              FNN_units=FNN_UNITS,
              emb_dim=EMB_DIM,
              nb_filter=FILTERS,
              nb_classes = NB_CLASSES,
              dropout_rate=DROPOUT_RATE
              )

In [501]:
cnn.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

In [502]:
cnn.fit(x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS)

Epoch 1/3
2189/2189 [==============================] - 196s 89ms/step - loss: 0.4961 - accuracy: 0.7385
Epoch 2/3
2189/2189 [==============================] - 196s 90ms/step - loss: 0.1706 - accuracy: 0.9356
Epoch 3/3
2189/2189 [==============================] - 196s 89ms/step - loss: 0.0589 - accuracy: 0.9773


In [503]:
result = cnn.evaluate(x_test,y_test,batch_size=BATCH_SIZE)
print(result)

938/938 [==============================] - 4s 4ms/step - loss: 0.6746 - accuracy: 0.7927
[0.6746255159378052, 0.7926552891731262]


In [504]:
def rXX(ss):
  if ss > 0.8:
    return 1
  else:
    return 0

In [505]:
inputText = "Steve jobs"
text = inputText[:8]
f = np.array([tokenizer.encode(text)])
l = [0 for i in range(8)]
for i in range(len(f[0])):
  l[i] = f[0][i]

fgg = np.array([l])
fgg
ta = cnn(fgg, training=False).numpy()[0][0]
rXX(ta)

1